### Path way setting

In [ ]:
# If storage the dataset in google dirve run this cell and the cell below
# set the environment for google drive
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [0]:
# mark the direction to Google Drive
!mkdir -p drive
!google-drive-ocamlfuse drive

### Package import

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as image
from os.path import join
import tensorflow as tf
tf.enable_eager_execution()
from tensorflow.keras.applications import Xception
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import GlobalAveragePooling2D
import tensorflow.contrib.eager as tfe
from matplotlib import pyplot as plt
from glob import glob
import random
import time

In [0]:
# if the device has a GPU, run the programm with GPU, otherwise run with CPU
device = "gpu:0" if tfe.num_gpus() else "cpu:0"

### Data Loading

In [0]:
# load decorded dataset 
npz_path = 'drive/Resized_Histo_50_20000.npz'  # set the path way of data
data = np.load(npz_path)

X = data['image']   # select the 'image' column in data to X
train_labels = data['labels']   # select the 'labels' column in data to train_labels

In [0]:
# split dataset into train, test and validation sets
from sklearn.model_selection import train_test_split

X, X_test, y, y_test = train_test_split(X, train_labels, test_size=0.1)
X_train, X_validation, y_train, y_validation = train_test_split(X, y, test_size=0.05)

### Model Building

In [0]:
# build the network with pre-trained Xception model
class Classifier(tf.keras.Model):
    
    def __init__(self, n_classes):
        super(Classifier, self).__init__()
        self.xception_layers = Xception(include_top=False, weights='imagenet')  
        self.pooling_layer = GlobalAveragePooling2D()
        self.dense_layer1 = tf.keras.layers.Dense(units=128, activation='relu')
        self.dense_layer2 = tf.keras.layers.Dense(units=64, activation='relu')
        self.dropout_layer1 = tf.keras.layers.Dropout(0.5)
        self.dense_layer3 = tf.keras.layers.Dense(units=n_classes)
        
    def call(self, inputs):
        result = self.xception_layers(inputs)
        result = self.pooling_layer(result)
        result = self.dense_layer1(result)
        result = self.dense_layer2(result)
        result = self.dropout_layer1(result)
        result = self.dense_layer3(result)
        return result

In [0]:
# build the network with pre-trained Inception model
class Classifier(tf.keras.Model):
    
    def __init__(self, n_classes):
        super(Classifier, self).__init__()
        self.xception_layers = InceptionV3(include_top=False, weights='imagenet')  
        self.pooling_layer = GlobalAveragePooling2D()
        self.dense_layer1 = tf.keras.layers.Dense(units=20, activation='relu')
        self.dense_layer2 = tf.keras.layers.Dense(units=20, activation='relu')
        self.dense_layer3 = tf.keras.layers.Dense(units=n_classes)
        
    def call(self, inputs):
        result = self.xception_layers(inputs)
        result = self.pooling_layer(result)
        result = self.dense_layer1(result)
        result = self.dense_layer2(result)
        result = self.dense_layer3(result)
        return result

In [0]:
# build shallow convolution neural network
class Classifier(tf.keras.Model):
    
    def __init__(self, n_classes):
        super(Classifier, self).__init__()
        self.convolution_layer1 = tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu')
        self.convolution_layer = tf.keras.layers.Conv2D(96, (3, 3), activation='relu')
        self.pooling_layer = tf.keras.layers.MaxPool2D(pool_size=(2, 2))
        self.dropout_layer1 = tf.keras.layers.Dropout(0.25)
        self.flatten_layer = tf.keras.layers.Flatten()
        self.dense_layer1 = tf.keras.layers.Dense(units=128, activation='relu')
        self.dropout_layer2 = tf.keras.layers.Dropout(0.5)  
        self.dense_layer2 = tf.keras.layers.Dense(units=n_classes)
        
    def call(self, inputs):
        result = self.convolution_layer(inputs)
        result = self.pooling_layer(result)
        result = self.dropout_layer1(result)
        result = self.flatten_layer(result)
        result = self.dense_layer1(result)
        result = self.dropout_layer2(result)
        result = self.dense_layer2(result)
        return result

### Model Initiation

In [0]:
# parameters setting
batch_size = 100
n_epochs = 50
learning_rate = 0.0001

train_loss_history = []
validation_loss_history = []

In [0]:
# slice the dataset and shuffle it

train_images_dataset = tf.data.Dataset.from_tensor_slices(X_train)
train_labels_dataset = tf.data.Dataset.from_tensor_slices(y_train)
train_dataset = tf.data.Dataset.zip((train_images_dataset, train_labels_dataset))
train_dataset = train_dataset.shuffle(buffer_size=len(X_train)).batch(batch_size)

In [0]:
# initiate the model and set the model with optimizer

x_classifier = Classifier(n_classes=2)
optimizer = tf.train.AdamOptimizer(learning_rate) 

### Model Training

In [ ]:
# training process

with tf.device(device):
    losscount = 0
    for epoch in range(n_epochs):
        start = time.time()
        for batch, (images, labels) in enumerate(train_dataset):
            with tf.GradientTape() as tape:
                # Compute logits
                logits = x_classifier(images)
                
                # Coumpute loss
                xe_loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=logits))                
                train_loss_history.append(xe_loss.numpy())
            # Compute gradient and apply gradients
                                         
            grads = tape.gradient(xe_loss, x_classifier.variables)
            optimizer.apply_gradients(zip(grads, x_classifier.variables),global_step=tf.train.get_or_create_global_step())
            
            
            if batch % 10 == 0:
                print('\rEpoch: {}, Batch: {}, Loss: {}'.format(epoch, batch, train_loss_history[-1]), end='')

        # Compute validation set    
        logits_v = x_classifier(tf.constant(X_validation))
        xe_loss_v = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y_validation, logits=logits_v))
        validation_loss_history.append(xe_loss_v.numpy()) 
         
        # early stop    
        if len(validation_loss_history) >= 5:
            if xe_loss_v > np.array(validation_loss_history[-1:-6:-1]).mean():
                losscount +=1
            else:
                losscount = 0
                
        if losscount > 3:
            break
       
        print('\rEpoch: {}, Batch: {}, Train Loss: {}, Count: {}'.format(epoch, batch, train_loss_history[-1],losscount))
        print('\rEpoch: {}, Batch: {}, Validation Loss: {}, Count: {}'.format(epoch, batch, validation_loss_history[-1],losscount))
        print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

### Result and Evaluation

In [ ]:
# make plot for train loss
plt.figure(figsize=(6,4))
plt.plot(train_loss_history)
plt.xlabel('Iterations', fontsize=14)
plt.ylabel('loss', fontsize=14);

In [ ]:
# make plot for validation loss
plt.figure(figsize=(6,4))
plt.plot(validation_loss_history)
plt.xlabel('Epoch', fontsize=14)
plt.ylabel('loss', fontsize=14);

In [0]:
# apply softmax to calculatehe probabilities for each classes
logits = x_classifier(tf.constant(X_test))
y_pred = tf.nn.softmax(logits).numpy()

In [ ]:
# calculate accuracy
accuracy_result_1=[]
for i in range(len(y_test)):
    accuracy_result_1.append(y_test[i] - np.argmax(y_pred,axis=1)[i])
accuracy_result_1.count(0)/len(y_test)

In [0]:
# predict rsult 
y_pred_result =  np.argmax(y_pred,axis=1)

In [0]:
# plot the confusion matrix

from sklearn.metrics import confusion_matrix
def plot_confusion_matrix(cm, classes,
    normalize=False,
    title='Confusion matrix',
    cmap=plt.cm.Blues):
    import itertools
    """
     This function prints and plots the confusion matrix.
     Normalization can be applied by setting `normalize=True`.
     """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')
    print(cm)
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    #plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes)
    plt.yticks(tick_marks, classes)
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),horizontalalignment="center",color="white" if cm[i, j] > thresh else "black")
    plt.tight_layout()
    plt.ylabel('True Diagnosis')
    plt.xlabel('Predicted Diagnosis')

In [ ]:
# plot for test set
cm=confusion_matrix(y_test, y_pred_result,np.unique(y_test))
plot_confusion_matrix(cm,np.unique(y_test) )

In [ ]:
# make ROC plot and calcualte AUC
import sklearn.metrics as metrics
# calculate the fpr and tpr for all thresholds of the classification

fpr, tpr, threshold = metrics.roc_curve(y_test, y_pred[:,1])
#fpr_n, tpr_n, threshold_n = metrics.roc_curve(y_test, y_pred[:,1])

# Calculate AUC area and print
roc_auc = metrics.auc(fpr, tpr)

print('Auc value based on Xception:', roc_auc)

# Plot ROC curve for test dataset
import matplotlib.pyplot as plt
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'Xception, AUC = %0.2f' % roc_auc)

plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()